# py2mac library usage example

In [ ]:
from py2mac.utils import check_access
from py2mac.application import get_running_applications
from py2mac.json_utils import json_serializable
from py2mac.data_extraction import clean_ui_tree, flatten_ui_tree
import json

import ipywidgets
from IPython.display import display

In [ ]:
# check_access verifies if the library can access macos accessibility interface
check_access()

### Selecting application

In [ ]:
# Get all currently running applications
applications = get_running_applications()

# Show app selector
app_widget = ipywidgets.Dropdown(
    options=list(sorted(applications, key=lambda app: app.localized_name)),
    description='Selected application:',
    disabled=False,
)
display(app_widget)

In [ ]:
app = app_widget.value
print(f"Selected app: {app}")

### Retrieving UI status

In [ ]:
# (optional) provide list of included attributes and actions
# will include all if None (slows down the processing)

# included_attributes=['AXTitle','AXRole','AXValue','AXDescription','AXPlaceholderValue','AXSelected','AXSubrole','AXRoleDescription']
# included_actions=['AXPress']
included_attributes = None
included_actions = None

component_tree = app.root_ui_element.asdict(
    included_attributes=included_attributes, included_actions=included_actions,
    include_empty_attributes=True
)

In [ ]:
component_tree

In [ ]:
cleaned_component_tree = clean_ui_tree(component_tree)

In [ ]:
cleaned_component_tree

In [ ]:
flat_cleaned_component_tree = flatten_ui_tree(cleaned_component_tree)

In [ ]:
flat_cleaned_component_tree

In [ ]:
# use json_serializable to dump 
json.dumps(json_serializable(flat_cleaned_component_tree))

## UI Interaction Example

Py2Mac can interact with UI components by:
* Triggering actions
* Setting attributes

### Select UI component

In [ ]:
# Show UI element selector
ui_component_widget = ipywidgets.Dropdown(
    options=sorted(flat_cleaned_component_tree, key=lambda el: el.get("id", "")),
    description='Selected component:',
    disabled=False,
)
display(ui_component_widget)

In [ ]:
ui_component_dict = ui_component_widget.value
ui_component_dict

In [ ]:
ui_element = app.get_ui_element(ui_component_dict['id'])

### Trigger action

In [ ]:
# Show UI action selector
ui_action_widget = ipywidgets.Dropdown(
    options=ui_component_dict['actions'],
    description='Selected action:',
    disabled=False,
)
display(ui_action_widget)

In [ ]:
# Call selected UI action
ui_action = getattr(ui_element, ui_action_widget.value)
ui_action()

# We use getattr here to select action dynamically, but it can be called directly as e.g. ui_element.AXPress()

### List attributes

In [ ]:
ui_element.attributes

In [ ]:
ui_element.settable_attributes

### Set attributes

In [ ]:
# Show UI widget attribute selector
ui_attr_widget = ipywidgets.Dropdown(
    options=ui_element.settable_attributes,
    description='Selected attribute:',
    disabled=False,
)
display(ui_attr_widget)

In [ ]:
ui_attr = ui_attr_widget.value
# Show UI widget attribute value input
ui_attr_value_widget = ipywidgets.Text(
    value=getattr(ui_element, ui_attr),
    description='Attribute value:',
    disabled=False,
)
display(ui_attr_value_widget)

In [ ]:
# Set attribute value
setattr(ui_element, ui_attr, ui_attr_value_widget.value)